In [1]:
#Exercise 3: Stochastic Programming
import gurobipy as gp
from gurobipy import GRB
import numpy as np

In [2]:
capacity = [18, 24, 27, 22, 31]
varcost = [[1675, 400, 685, 1630, 1160, 2800], 
           [1460, 1940, 970, 100, 495, 1200], 
           [1925, 2400, 1425, 500, 950, 800], 
           [380, 1355, 543, 1045, 665, 2321], 
           [922, 1646, 700, 508, 311, 1797]]
fixcost = [7650, 3500, 5000, 4100, 2200]

locations = 5
customers = 6

mean_demand = [10, 8, 14, 6, 7, 11]
std_demand = [x/2 for x in mean_demand]

In [3]:
#Using Monte Carlo sampling strategy
def demand_gen(n, mean, std): #generate random integer demand matrix = sample size x customers  e.g. 20/50/100 x 6
    import numpy as np
    d = np.empty((len(mean),n))
    for i in range(len(mean)):
        d[i,:] = [int(x) for x in np.random.normal(mean[i], std[i], size = n)]
    return np.transpose(d)

In [4]:
#Part a)
# Sample Average Approximation
# Using Monte Carlo sampling strategy

In [5]:
#define fuction to solve SAA problem
def SAA_MILP (n, demand):
    # Warehouse Location Problem

    model = gp.Model("MILP Warehouse Location Problem")

    # Create variables
    x = {} # transport qty of customer j served by location i in scenario s
    y = {} # binary variables representing decision to open location or not

    for s in range(n):
        for i in range(locations):
            for j in range(customers):
                x[s,i,j] = model.addVar(obj=varcost[i][j]* (1/n)) #p(s) = 1/n

    for i in range(locations):
        y[i] = model.addVar(vtype = GRB.BINARY, obj=fixcost[i])
    
    #fc = sum(y[i]*fixcost[i] for i in range(locations))
    #vc = np.mean( [sum([x[s,i,j]*varcost[i][j] for i in range(locations) for j in range(customers)]) for s in range(n)] )
    #model.setObjective(fc+vc)

    #Capacity constraint    
    cap_constr = {}
    for s in range(n):
        for i in range(locations):
            cap_constr[s,i] = model.addConstr(sum(x[s,i,j] for j in range(customers)) <= y[i]*capacity[i])

    # Demand constraints
    demand_constr = {} 
    for s in range(n): 
        for j in range(customers):
            demand_constr[s,j] = model.addConstr(sum(x[s,i,j] for i in range(locations)) >= demand[s,j])


    model.optimize()


    print("Objective: " + str(model.objVal))

    print("Solution:")
    print([y[i].x for i in range(locations)])
    
    return (model.objVal, [y[i].x for i in range(locations)])

In [6]:
#Solve with n = 20 / 50 / 100
obj_list = []
y_list = []

In [7]:
# sampling size = 20
np.random.seed(1)

#generate demand
demand20 = demand_gen(20, mean_demand, std_demand)
print(demand20)
print('Mean: ', [np.mean(demand20[:,j]) for j in range(customers)])
print('Std: ', [np.std(demand20[:,j]) for j in range(customers)])
print('\n')
(obj, y) = SAA_MILP(20,demand20)
obj_list.append(obj)
y_list.append(y)

[[18.  3. 12.  3.  6.  8.]
 [ 6. 12.  7.  9.  6. 17.]
 [ 7. 11.  8.  7.  7. 13.]
 [ 4. 10. 25.  5.  8. 14.]
 [14. 11. 14.  7.  7.  4.]
 [-1.  5.  9.  5.  7. 11.]
 [18.  7. 15.  9.  4. 15.]
 [ 6.  4. 28. 10.  8.  5.]
 [11.  6. 14. 12.  7.  9.]
 [ 8. 10. 18.  1. 10. 11.]
 [17.  5. 16.  1. 11.  3.]
 [ 0.  6. 11.  4.  7. 12.]
 [ 8.  5.  6.  6.  5. 15.]
 [ 8.  4. 11.  8.  4.  6.]
 [15.  5. 12.  6.  8. 12.]
 [ 4.  7. 18.  0.  7.  3.]
 [ 9.  3. 19.  5.  5. 10.]
 [ 5.  8. 20.  8.  7.  2.]
 [10. 14. 15.  6.  4. 17.]
 [12. 10. 20.  8.  9. 13.]]
Mean:  [8.95, 7.3, 14.9, 6.0, 6.85, 10.0]
Std:  [5.352335938634644, 3.1638584039112754, 5.638262143604179, 3.0495901363953815, 1.851350858157362, 4.669047011971501]


Using license file C:\Users\hadao\gurobi.lic
Academic license - for non-commercial use only
Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (win64)
Optimize a model with 220 rows, 605 columns and 1300 nonzeros
Model fingerprint: 0xc1ba792d
Variable types: 600 continuous, 5 integer (5 binary)


In [8]:
# sampling size = 50
np.random.seed(1)

#generate demand
demand50 = demand_gen(50, mean_demand, std_demand)
print('Mean: ', [np.mean(demand50[:,j]) for j in range(customers)])
print('Std: ', [np.std(demand50[:,j]) for j in range(customers)])
print('\n')
(obj, y) = SAA_MILP(50,demand50)
obj_list.append(obj)
y_list.append(y)

Mean:  [9.42, 8.1, 14.02, 6.22, 6.3, 10.88]
Std:  [4.783680591343866, 3.1192947920964444, 6.143256465426134, 2.8163096420670795, 3.8065732621348563, 4.921950832749145]


Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (win64)
Optimize a model with 550 rows, 1505 columns and 3250 nonzeros
Model fingerprint: 0x8c3fbc9c
Variable types: 1500 continuous, 5 integer (5 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+01]
  Objective range  [2e+00, 8e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+01]
Presolve removed 8 rows and 40 columns
Presolve time: 0.00s
Presolved: 542 rows, 1465 columns, 3170 nonzeros
Variable types: 1460 continuous, 5 integer (5 binary)

Root relaxation: objective 4.168131e+04, 462 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 41681.3105    0    4          - 41681.3105      -     -    0s
H    0 

In [9]:
# sampling size = 100
np.random.seed(1)

#generate demand
demand100 = demand_gen(100, mean_demand, std_demand)
print('Mean: ', [np.mean(demand100[:,j]) for j in range(customers)])
print('Std: ', [np.std(demand100[:,j]) for j in range(customers)])
print('\n')
(obj, y) = SAA_MILP(100,demand100)
obj_list.append(obj)
y_list.append(y)

Mean:  [9.81, 8.13, 13.52, 5.46, 6.8, 11.19]
Std:  [4.358199169381776, 3.7058197473703443, 7.003541961036572, 3.141400961354663, 3.5355339059327378, 5.709106760255934]


Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (win64)
Optimize a model with 1100 rows, 3005 columns and 6500 nonzeros
Model fingerprint: 0x2ec42ef6
Variable types: 3000 continuous, 5 integer (5 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+01]
  Objective range  [1e+00, 8e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+01]
Presolve removed 23 rows and 115 columns
Presolve time: 0.01s
Presolved: 1077 rows, 2890 columns, 6270 nonzeros
Variable types: 2885 continuous, 5 integer (5 binary)

Root relaxation: objective 4.228029e+04, 875 iterations, 0.01 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 42280.2902    0    5          - 42280.2902      -     -    0s
H  

In [10]:
print("Comparison:")
import pandas as pd

solution = pd.DataFrame(index=['Opened WH', 'Objective value'] , columns=['n=20', 'n=50', 'n=100'])
solution.iloc[0] = y_list
solution.iloc[1] = obj_list
print(solution)

Comparison:
                                       n=20                       n=50  \
Opened WH        [1.0, -0.0, 1.0, 1.0, 1.0]  [1.0, 0.0, 1.0, 1.0, 1.0]   
Objective value                     47144.4                    47806.1   

                                      n=100  
Opened WH        [1.0, -0.0, 1.0, 1.0, 1.0]  
Objective value                     47767.5  


The decisions to open WH in three sample size cases are the same while the objective values are slightly different (because of sampling error).

In [11]:
#Part b)
#Solve n=100 with Benders decomposition

In [12]:
def subproblem(model, where):
    if where == GRB.Callback.MIPSOL:
        v_y = model.cbGetSolution(model._y)
        #print(v_y)
        LB = model.cbGetSolution(model._z)
        print('Current LB: ', LB, '\n')
        print(model.cbGet(GRB.Callback.MIPSOL_OBJBND))
        print(model.cbGet(GRB.Callback.MIPSOL_OBJBST))
        
        bsp = gp.Model("Subproblem") 
        x = {}
        for s in range(n):
            for i in range(locations):
                for j in range(customers):
                    x[s,i,j] = bsp.addVar(obj=varcost[i][j]*(1/n)) #p(s) = 1/n
        
        #unserved demand
        u = {}
        add_cost = 1000000
        for s in range(n):
            for j in range(customers):
                u[s,j] = bsp.addVar( obj = add_cost*(1/n)) #p(s) = 1/n
        
        demand_constr = {} 
        for s in range(n):        
            for j in range(customers):
                demand_constr[s,j] = bsp.addConstr(sum(x[s,i,j] for i in range(locations)) + u[s,j] >= demand[s,j])
        
        cap_constr = {}
        for s in range(n):        
            for i in range(locations):
                cap_constr[s,i] = bsp.addConstr(sum(x[s,i,j] for j in range(customers)) <= v_y[i]* capacity[i])
        
        bsp.optimize()

        
        #update ub and lb
        fc = 0
        for i in range(locations):
            if sum(round(x[s,i,j].x) for j in range(customers) for s in range(n)) > 0:
                fc+=fixcost[i]
        
        if bsp.objVal + fc > model.cbGet(GRB.Callback.MIPSOL_OBJBND):
            v = np.zeros((n,customers)) #dual of demand constraints
            for s in range(n):            
                for j in range(customers):
                    v[s,j] = demand_constr[s,j].pi #get dual value
            
            w = np.zeros((n,locations)) #dual of capacity constraint
            for s in range(n):            
                for i in range(locations):
                    w[s,i] = cap_constr[s,i].pi

            cm = np.zeros(locations) #coefficient of y in master problem
            for i in range(locations):
                cm[i] = fixcost[i]+ sum(capacity[i] * w[s,i] for s in range(n))
            

            model.cbLazy(model._z  >= sum( demand[s,j] * v[s,j] for j in range(customers) for s in range(n)) +   
                         sum(cm[i] * model._y[i] for i in range(locations)))


        bsp.dispose()
                       

In [13]:
transport = np.zeros((locations, customers)) # to save result of x from subproblem
# Masterproblem
                     
m = gp.Model("Benders Masterproblem")

y = {}
z = m.addVar(obj=1)                                                                           
for i in range(locations):
    y[i] = m.addVar(vtype=GRB.BINARY)
    
n = 100
demand = demand100

m.Params.lazyConstraints = 1
m._y = y
m._z = z
m.optimize(subproblem)
    
    
print("Objective: " + str(m.objVal))

print([y[i].x for i in range(locations)])

Changed value of parameter lazyConstraints to 1
   Prev: 0  Min: 0  Max: 1  Default: 0
Changed value of parameter TimeLimit to 5400.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (win64)
Optimize a model with 0 rows, 6 columns and 0 nonzeros
Model fingerprint: 0xc4144e5c
Variable types: 1 continuous, 5 integer (5 binary)
Coefficient statistics:
  Matrix range     [0e+00, 0e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [0e+00, 0e+00]
Current LB:  0.0 

-1e+100
0.0
Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (win64)
Optimize a model with 1100 rows, 3600 columns and 6600 nonzeros
Model fingerprint: 0x685a1ed8
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+04]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 3e+01]
Presolve removed 1100 rows and 3600 columns
Presolve time: 0.01s
Presolve: All rows and columns removed
Iteration    Objective  

Current LB:  34778.920000000006 

32812.608697476884
34778.920000000006
Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (win64)
Optimize a model with 1100 rows, 3600 columns and 6600 nonzeros
Model fingerprint: 0x13aeaa30
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+04]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 3e+01]
Presolve removed 123 rows and 715 columns
Presolve time: 0.01s
Presolved: 977 rows, 2885 columns, 5193 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   2.752500e+03   0.000000e+00      0s
     656    2.8817480e+04   0.000000e+00   0.000000e+00      0s

Solved in 656 iterations and 0.02 seconds
Optimal objective  2.881748000e+04
Current LB:  47767.479999999996 

32812.608697476884
47767.479999999996
Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (win64)
Optimize a model with 1100 rows, 3600 columns and 6600 nonzeros
Model fingerprint: 0x13aeaa30
Coefficient sta

We obtain same result from MILP and Benders decomposition.